In [1]:
import numpy as np

In [2]:
data = np.load('sdss_galaxy_colors.npy')

In [3]:
data[0]

(19.84132, 19.52656, 19.46946, 19.17955, 19.10763, b'QSO', 0.539301, 6.543622e-05)

In [4]:
data['u']

array([19.84132, 19.86318, 19.97362, ..., 19.82667, 19.98672, 18.00024])

In [5]:
data.shape

(50000,)

In [6]:
data.dtype.names

('u', 'g', 'r', 'i', 'z', 'spec_class', 'redshift', 'redshift_err')

In [7]:
def get_features_targets(data):
    features = np.zeros((data.shape[0],4))
    features[:,0] = data['u'] - data['g']
    features[:,1] = data['g'] - data['r']
    features[:,2] = data['r'] - data['i']
    features[:,3] = data['i'] - data['z']

    targets = data['redshift']

    return features, targets 

In [8]:
features, targets = get_features_targets(data)

In [9]:
from sklearn.tree import DecisionTreeRegressor

In [10]:
dtr = DecisionTreeRegressor()
dtr.fit(features, targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [22]:
predictions = dtr.predict(features)

In [23]:
def median_diff(predicted, actual):
    return np.median(np.abs(predicted - actual))

In [24]:
median_diff(predictions, targets)

0.0

In [29]:
def validate_model(model, features, targets):
    split = features.shape[0]//2
    train_features = features[:split]
    train_targets = targets[:split]
    test_features = features[split:]
    test_targets = targets[split:]

    model.fit(train_features, train_targets)

    predictions = model.predict(test_features)

    return median_diff(test_targets, predictions)

In [30]:
validate_model(DecisionTreeRegressor(), features, targets)

0.021892724999999995